<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/PNNPPV/notebooky/cv07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. cvičenie - Fine-tuning a vizualizácia

Na dnešnom cvičení si prejdeme metódu tzv. fine-tuningu, ktorá spadá pod tzv. metódy transfer learningu. Túto úlohu vyskúšate na Google Cloude. V druhej časti cvičenia si skúsime zobraziť ktoré časti obrázku sú pre sieť dôležité na klasifikáciu.

## 1. Úloha

Túto úlohu vypracujte na cloude. Pripraviť si ju môžete u seba, alebo v Colabe. Skúste si ju však spustiť aj na cloude.

V tejto úlohe budeme trénovať sieť s predtrénovanými váhami tzv. fine-tuning alebo transfer learning. Predtrénované modely prevezmeme z [keras applications](https://keras.io/applications/) na datasete cats vs. dogs, ktorý stiahnete napríklad tu:

```
http://files.fast.ai/data/dogscats.zip
```

Pre prácu s datasetom použite ImageDataGenerator a jeho metódu flow_from_directory.


Po načítaní modelu máme tri možnosti, buď ponecháme vrstvy ako trénovateľne, všetky zamrazíme, alebo zamrazíme len niektoré začiatočne. Zmrziť vrstvy môýžeme napr.:
```python
xception = keras.applications.xception.Xception(include_top = False)
for layer in xception.layers:
    layer.trainable = False
```

Model použite "bez vrcholu" (include_top = False). A pridajte ho ako "vrstvu" do sekvenčného modelu a po ňom realizujte globálny pooling a kratšiu plne prepojenú sieť na klasifikáciu. Otestujte ako ide tréning pre všetky tri možnosti trénovania častí siete.


## Vizualizácia častí obrazu dôležitých pre sieť

Teraz si skúsime overiť ktoré časti v obraze sú pre klasifikáciu dôležité. To spravíme tak, že si vezmeme testovací obrázok zistíme ako sieť klasifikuje. Potom do obrázka budeme vkladať na rôzne miesta čierny štvorec. Potom zistíme ako veľmi sa zmenila predpoveď modelu. Z týchto zmien potom spravíme heatmapu.

In [0]:
import keras
import cv2
import numpy as np
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import matplotlib.pyplot as plt

Toto spravíme znova s predtrénovaným modelom.

In [0]:
resnet = keras.applications.resnet.ResNet50()

Načítame si testovací obrázok a zmeníme jeho rozmery.

In [0]:
test_imgs = []
!wget https://pixnio.com/free-images/2017/06/08/2017-06-08-13-53-59-900x576.jpg
test_imgs.append(cv2.resize(cv2.imread('2017-06-08-13-53-59-900x576.jpg'),(224,224)))
!wget https://storage.needpix.com/rsynced_images/diver-1881751_1280.jpg
test_imgs.append(cv2.resize(cv2.imread('diver-1881751_1280.jpg'),(224,224)))
!wget https://cdn.pixabay.com/photo/2017/09/22/23/24/white-stork-2777489_960_720.jpg
test_imgs.append(cv2.resize(cv2.imread('white-stork-2777489_960_720.jpg'),(224,224)))
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Croatia_police_van_%2804%29.JPG/800px-Croatia_police_van_%2804%29.JPG
test_imgs.append(cv2.resize(cv2.imread('800px-Croatia_police_van_(04).JPG'),(224,224)))


Pred vstupom do siete ešte aplikujeme predspracovanie, aby to korešpondovalo s natrénovanou sieťou. **Treba si taktiež dať pozor na to či je formát v BGR ako to má OpenCV, alebo RGB!**

In [0]:
pred = resnet.predict(preprocess_input(test_imgs[0])[None, :, :, :])
print(np.argmax(pred[0]))
print(decode_predictions(pred)[0])

Túto funkciu napíšte tak aby vrátila heatmapu veľkosti $rectangle\_num \times rectangle\_num$. Jednotlivé elementy heatmapy budú reprezentovať to ako veľmi sa zmení predikcia pre pôvodnú triedu obrázku ak na korešpondujúce miesto v obrázku pridáme čierny štvorec veľkosti $rectangle\_size \times rectangle\_size$.

Pri implementácii skúste využiť to, že metóda model.predict dokáže spracovať viacero obrázkov naraz ak ich poukladáte "na seba" podobne ako pri generovaní batchu.

In [0]:
def generate_heatmap(img, model, rectangle_num, rectangle_size):
  heatmap = np.zeros([rectangle_num, rectangle_num])
  return heatmap

Tento kód by mal zobraziť heatmapy pre vybrané obrázky.

In [0]:
for test_img in test_imgs:
  hmap = generate_heatmap(test_img, resnet, 20, 9)
  plt.imshow(hmap,cmap='gray')
  plt.show()
  plt.imshow(test_img[:,:,::-1])
  plt.show()